
## Advanced Feature Engineering & "Alpha" Signals

###  Objective
In this notebook, we transform raw data into **Financial Intelligence**.
Standard machine learning models (like XGBoost) are excellent at finding patterns, but they struggle to learn complex mathematical ratios (like $A / B$) efficiently. By explicitly engineering these signals, we "linearize" the risk for the model.

###  Domain Research: The Physics of Credit Risk
Our engineering strategy is based on the fundamental economics of borrower behavior:

#### 1. The "Skin in the Game" Theory (LTV)
* **Concept:** Borrowers rarely default strategically if they have significant equity in the property. Default risk spikes when the loan amount approaches the property value.
* **The Signal:** **Loan-to-Value (LTV)** = $\frac{\text{Loan Amount}}{\text{Property Value}}$.
* **Hypothesis:** A sharp non-linear increase in risk is expected when $LTV > 80\%$ (the standard PMI threshold).

#### 2. The "Burden" Theory (LTI & DTI)
* **Concept:** Income is "fuel," but the Loan is the "burn rate." A high earner can still be insolvent if their debt load is massive.
* **The Signal:** **Loan-to-Income (LTI)** = $\frac{\text{Loan Amount}}{\text{Annual Income}}$.
* **Hypothesis:** High LTI (> 5.0) indicates fragility to income shocks.

#### 3. The "Double Jeopardy" Interaction
* **Concept:** Risk factors are rarely additive; they are multiplicative.
* **The Signal:** A borrower with **Low Credit** might be fine. A borrower with **High LTV** might be fine. But a borrower with **BOTH** is exponentially riskier.
* **Engineering:** We will create specific boolean flags to capture these dangerous intersections.

In [1]:
# ---------------------------------------------------------
# Imports & Load Data
# ---------------------------------------------------------
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import joblib 

# Set Style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

# Load the Preprocessed Data (Output of Notebook 02)
input_path = '../data/processed/02_preprocessed_data.csv'

if os.path.exists(input_path):
    df = pd.read_csv(input_path)
    print(f"Loaded Clean Data. Shape: {df.shape}")
else:
    print(" Error: 02_preprocessed_data.csv not found. Run Notebook 02 first.")

df.head(3)

Loaded Clean Data. Shape: (47728, 25)


,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,business_or_commercial,loan_amount,term,Neg_ammortization,...,income,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Status,dtir1
0,cf,Male,nopre,type2,p1,l1,b/c,206500,360.0,not_neg,...,4980.0,EQUI,552,EXP,55-64,to_inst,74.641577,North,1,39.0
1,cf,Male,pre,type1,p1,l1,nob/c,406500,360.0,neg_amm,...,9480.0,EXP,834,CIB,35-44,to_inst,80.019685,south,0,46.0
2,cf,Male,nopre,type1,p4,l1,nob/c,456500,360.0,not_neg,...,11880.0,EXP,587,CIB,45-54,not_inst,69.376900,North,0,42.0




We now create the derived features based on our "Economic Physics" research.

**1. Loan-to-Income (LTI):**
$$LTI = \frac{\text{Loan Amount}}{\text{Annual Income}}$$
* *Why?* Measures the "Burn Rate" of the borrower's finances.
* *Logic:* If Income is 0 (edge case), we cap the ratio to avoid infinity.

**2. Disposable Income Proxy:**
$$\text{Disposable} = \text{Income} - (\text{Loan Amount} \times 0.005)$$
* *Why?* A \$500k loan costs roughly \$2,500/month (at ~6% interest/30yrs). This feature estimates "Cash Left to Eat."

**3. "Double Jeopardy" Flags:**
* **Risky LTV:** $1$ if $LTV > 80\%$ (Standard PMI Threshold).
* **Risky Credit:** $1$ if $Credit Score < 650$ (Subprime Threshold).
* **Double Risk:** $1$ if **BOTH** are true. This is the "Toxic Combination" we want the model to isolate.

In [2]:
# ---------------------------------------------------------
#  Executing Feature Engineering
# ---------------------------------------------------------

# 1. Loan-to-Income (LTI)
# Handle division by zero (though Income imputed to Median, let's be safe)
df['LTI'] = df['loan_amount'] / df['income'].replace(0, 1)

# 2. Disposable Income Proxy
# Estimate Monthly Payment approx = Loan * 0.005 (Simulated Interest)
df['est_monthly_payment'] = df['loan_amount'] * 0.005
df['disposable_income'] = df['income'] - df['est_monthly_payment']

# 3. Interaction Flags (The "Cliffs")
# Standard Bank Thresholds: LTV > 80 is risky, Score < 650 is subprime
df['risky_ltv'] = (df['LTV'] > 80).astype(int)
df['risky_credit'] = (df['Credit_Score'] < 650).astype(int)

# The "Double Jeopardy" Feature
df['double_risk'] = df['risky_ltv'] * df['risky_credit']

# 4. Debt-to-Income (Clean existing dtir1)
# Some rows might still be NaN if the original data was messy
df['dtir1'] = df['dtir1'].fillna(df['dtir1'].median())

print(" Engineered 'Alpha' Signals:")
print(df[['loan_amount', 'income', 'LTI', 'disposable_income', 'double_risk']].head())

# Validation: Check correlations of new features with Status
new_feats = ['LTI', 'disposable_income', 'double_risk', 'Status']
print("\n🔍 Correlation with Default:")
print(df[new_feats].corr()['Status'].sort_values(ascending=False))

 Engineered 'Alpha' Signals:
   loan_amount   income        LTI  disposable_income  double_risk
0       206500   4980.0  41.465863             3947.5            0
1       406500   9480.0  42.879747             7447.5            0
2       456500  11880.0  38.425926             9597.5            0
3       706500  10080.0  70.089286             6547.5            0
4       346500   5040.0  68.750000             3307.5            0

🔍 Correlation with Default:
Status               1.000000
LTI                  0.153414
double_risk         -0.019806
disposable_income   -0.041391
Name: Status, dtype: float64


### Categorical Encoding (Preparing for AI)

We need to convert text variables (e.g., `Region`, `Gender`) into numerical format.

**Why Label Encoding?**
* **Traditional Method:** One-Hot Encoding creates a new column for every category (e.g., `Region_North`, `Region_South`). This leads to "The Curse of Dimensionality" (too many columns, sparse data).
* **Our Method:** **Label Encoding** (0, 1, 2, 3).
    * *For XGBoost:* It handles ordinal integers natively.
    * *For PyTorch:* This is the required input format for **Entity Embedding Layers**. The network will learn that "0" and "1" are distinct vectors.

**Production Note:** We must save these `LabelEncoders`. When a live user sends "North" to the API, we need to convert it to the exact same number (e.g., `2`) that the model learned during training.

In [3]:
# ---------------------------------------------------------
#  Label Encoding & Artifact Saving
# ---------------------------------------------------------
from sklearn.preprocessing import LabelEncoder

# Initialize a dictionary to store our encoders
# We will need this dictionary for the FastAPI backend later
label_encoders = {}

# Select object columns (excluding Status if it's already int)
cat_cols = df.select_dtypes(include='object').columns

print(f" Encoding {len(cat_cols)} categorical features...")

for col in cat_cols:
    # Initialize and fit encoder
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    
    # Store it for production
    label_encoders[col] = le

# Save the encoders to the 'models' folder
# This is crucial for the "Reproducibility" of the project
joblib.dump(label_encoders, '../models/label_encoders.pkl')

print(" Encoding Complete.")
print(" Encoders saved to: ../models/label_encoders.pkl")

# Preview the "Digitized" Data
df.head()

 Encoding 17 categorical features...
 Encoding Complete.
 Encoders saved to: ../models/label_encoders.pkl


,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,business_or_commercial,loan_amount,term,Neg_ammortization,...,LTV,Region,Status,dtir1,LTI,est_monthly_payment,disposable_income,risky_ltv,risky_credit,double_risk
0,1,2,1,1,1,0,0,206500,360.0,2,...,74.641577,0,1,39.0,41.465863,1032.5,3947.5,0,1,0
1,1,2,2,0,1,0,1,406500,360.0,1,...,80.019685,3,0,46.0,42.879747,2032.5,7447.5,1,0,0
2,1,2,1,0,4,0,1,456500,360.0,2,...,69.376900,0,0,42.0,38.425926,2282.5,9597.5,0,1,0
3,1,1,2,0,1,0,1,706500,360.0,2,...,70.089286,0,0,40.0,70.089286,3532.5,6547.5,0,0,0
4,1,1,2,0,3,0,1,346500,360.0,2,...,79.109589,0,0,44.0,68.750000,1732.5,3307.5,0,0,0


In [4]:
# ---------------------------------------------------------
#  Save Engineered Data
# ---------------------------------------------------------
output_path = '../data/processed/03_engineered_data.csv'
df.to_csv(output_path, index=False)

print("Feature Engineering Complete.")
print(f"Final Data Saved to: {output_path}")
print(f"Final Dimensions: {df.shape}")

Feature Engineering Complete.
Final Data Saved to: ../data/processed/03_engineered_data.csv
Final Dimensions: (47728, 31)
